In [1]:
import pandas as pd, numpy as np
import json

1. Replicar 100 bps? -> **10 Basis points: dejar como parametro del backtest**
2. Determinar origin vs destination exchange -> **Kraken es origin, Bitfinex es destination**
3. Como replicar los niveles del origin en el destination (volumen)
4. Validar el método para determinar maker/taker fee: iterativo vs estático
5. Orden de llenado por tiempo de entrada.
6. Evolución de Orderbooks tipo bootstrap.

In [2]:
# load JSON object
f = open("orderbooks_05jul21.json")

# return JSON object as dictionary
orderbooks_data = json.load(f)
ob_bit = orderbooks_data['bitfinex']  # select specific exchange

# drop None keys
ob_bit = {i_key: i_value for i_key, i_value in ob_bit.items() if i_value is not None}

# create Dataframe and rearange columns
ob_bit = {i_ob: pd.DataFrame(ob_bit[i_ob])[['bid_size','bid','ask','ask_size']]
           if ob_bit[i_ob] is not None else None for i_ob in list(ob_bit.keys())}


ob_krak = orderbooks_data['kraken']  # select specific exchange

# drop None keys
ob_krak = {i_key: i_value for i_key, i_value in ob_krak.items() if i_value is not None}

# create Dataframe and rearange columns
ob_krak = {i_ob: pd.DataFrame(ob_krak[i_ob])[['bid_size','bid','ask','ask_size']]
           if ob_krak[i_ob] is not None else None for i_ob in list(ob_krak.keys())}

# Alineacion de timestamps

In [3]:
list(ob_krak.keys())[0], list(ob_bit.keys())[0], list(ob_bit.keys())[1], list(ob_bit.keys())[2], list(ob_bit.keys())[3]

('2021-07-05T13:06:49.495Z',
 '2021-07-05T13:06:46.571Z',
 '2021-07-05T13:06:47.918Z',
 '2021-07-05T13:06:49.414Z',
 '2021-07-05T13:06:51.077Z')

In [32]:
# filtro  para que destination sea despues de origin
aux = sum(pd.to_datetime(pd.Series((ob_bit.keys()))) < pd.to_datetime(pd.Series(ob_krak.keys()))[0])
[ob_bit.pop(key) for key in list(ob_bit.keys())[:aux]];

[]

# Origin exchange (Kraken)

In [21]:
# OB 1
df_krak = ob_krak[list(ob_krak.keys())[0]]
# midprice
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
display(df_krak.head(3))
mid_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074


28298.35

In [22]:
upper_krak = mid_krak*1.01
lower_krak = mid_krak*0.99
(lower_krak, upper_krak)

(28015.3665, 28581.333499999997)

In [23]:
bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]

In [24]:
[d.shape for d in [df_krak, bids_krak, asks_krak]]

[(100, 4), (100, 2), (100, 2)]

In [44]:
levels = pd.concat([bids_krak, asks_krak], axis=1)

In [110]:
levels

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074
3,28292.2,0.011,28305.5,0.123
4,28292.0,1.221,28307.3,0.204
...,...,...,...,...
95,28179.1,3.726,28430.4,0.001
96,28178.9,3.059,28431.0,0.001
97,28169.6,3.733,28431.8,3.840
98,28168.2,2.756,28432.1,3.032


# Destination exchange (Bitfinex)

In [15]:
# OB 1
df_bit = ob_bit[list(ob_bit.keys())[0]]
# midprice
mid_bit = (df_bit['bid'][0]+df_bit['ask'][0])/2
display(df_bit.head(3))
mid_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


28276.5

In [16]:
upper_bit = mid_bit*1.001
lower_bit = mid_bit*0.999
(lower_bit, upper_bit)

(28248.2235, 28304.776499999996)

In [17]:
bids_bit = df_bit[df_bit['bid']>lower_bit][['bid','bid_size']]
asks_bit = df_bit[df_bit['ask']<upper_bit][['ask','ask_size']]

In [18]:
[d.shape for d in [df_bit, bids_bit, asks_bit]]

[(25, 4), (21, 2), (25, 2)]

In [29]:
df_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000
3,0.003000,28268.0,28281.0,0.050000
4,1.581528,28266.0,28282.0,0.323781
5,2.000887,28265.0,28283.0,1.580312
6,1.309362,28264.0,28284.0,0.353892
7,0.133730,28263.0,28285.0,1.053899
8,0.596041,28262.0,28286.0,0.950994
9,0.188323,28260.0,28287.0,0.157200


# Añadir niveles a destination

In [101]:
new_df = df_bit.append(levels).reset_index(drop=True)

In [124]:
new_df['ask_size']

0      0.011063
1      0.006000
2      0.100000
3      0.050000
4      0.323781
         ...   
120    0.001000
121    0.001000
122    3.840000
123    3.032000
124    4.470000
Name: ask_size, Length: 125, dtype: float64

In [123]:
new_df

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28278.0,0.011063
1,0.350,28293.9,28279.0,0.006000
2,0.377,28293.5,28280.0,0.100000
3,0.011,28292.2,28281.0,0.050000
4,1.221,28292.0,28282.0,0.323781
...,...,...,...,...
120,3.726,28179.1,28430.4,0.001000
121,3.059,28178.9,28431.0,0.001000
122,3.733,28169.6,28431.8,3.840000
123,2.756,28168.2,28432.1,3.032000


In [106]:
new_df = pd.concat([new_df[['bid_size', 'bid']].sort_values('bid', ascending=False).reset_index(drop=True), 
                   new_df[['ask', 'ask_size']].sort_values('ask', ascending=True).reset_index(drop=True)],
                   axis=1)

In [125]:
new_df

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28278.0,0.011063
1,0.350,28293.9,28279.0,0.006000
2,0.377,28293.5,28280.0,0.100000
3,0.011,28292.2,28281.0,0.050000
4,1.221,28292.0,28282.0,0.323781
...,...,...,...,...
120,3.726,28179.1,28430.4,0.001000
121,3.059,28178.9,28431.0,0.001000
122,3.733,28169.6,28431.8,3.840000
123,2.756,28168.2,28432.1,3.032000


# Efectos en OB destination

In [128]:
#flags['maker' if bid<mid_bit else 'taker' for bid in levels['bid']]

In [129]:
levels.head(3)

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074


In [132]:
levels['bid_size'][0]

0.88

In [140]:
df_bit.head(6)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000
3,0.003000,28268.0,28281.0,0.050000
4,1.581528,28266.0,28282.0,0.323781
5,2.000887,28265.0,28283.0,1.580312


In [146]:
df_bit[(.88 - df_bit['ask_size'].cumsum())>0]

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000
3,0.003000,28268.0,28281.0,0.050000
4,1.581528,28266.0,28282.0,0.323781


In [138]:
(df_bit['ask_size'].cumsum() - .88)

0     -0.868937
1     -0.862937
2     -0.762937
3     -0.712937
4     -0.389156
5      1.191156
6      1.545048
7      2.598946
8      3.549940
9      3.707140
10     4.604367
11     8.020301
12     9.615977
13     9.879609
14    14.015022
15    14.882243
16    17.817491
17    18.883795
18    24.307846
19    24.418232
20    26.610706
21    32.126897
22    32.929496
23    33.133014
24    36.049183
Name: ask_size, dtype: float64